# Validación FACTURACIÓN

In [ ]:
import pandas as pd 
import numpy as np 
import os
from google.cloud import bigquery
from google.oauth2 import service_account
from datetime import datetime
from dateutil.relativedelta import relativedelta
import itertools

pd.set_option('display.max_columns', None)
# declaramos las credenciales al entorno de desarollo
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "..\\..\\bd\\0_docs\\datahub-deacero-adojeda.json"

query_str = """
SELECT
FORMAT_DATE('%Y-%m-%d', PARSE_DATE('%B.%Y', mes_anio)) AS MesAnio,
nom_grupo AS NombreGrupo,
nom_direccion AS NombreDireccion,
nom_subdireccion AS NombreSubdireccion,
nom_grupo_estadistico1 AS NombreGrupoEstadistico1,
nom_grupo_estadistico2 AS NombreGrupoEstadistico2,
nom_grupo_estadistico3 AS NombreGrupoEstadistico3,
sum(toneladas_facturadas) AS `Toneladas Facturadas`,
sum(toneladas_pvo) AS toneladas_pvo,
sum(toneladas_plan_ventas) AS toneladas_plan_ventas
FROM `datahub-deacero.mart_comercial.comercial` 
WHERE 
EXTRACT(YEAR FROM fecha) >= 2022
AND
EXTRACT(YEAR FROM fecha) < 2026
AND
nom_direccion NOT IN ('ABASTECIMIENTOS', 'IIDEA NACIONAL')   #('FILIALES') --> lo quité de la consulta duplica SUBDIRG3
#AND
#nom_subdireccion NOT IN  ('SERVICIOS LOGISTICOS DAL USA') --> lo quité de la consulta duplica SUBDIRG3
-- Descartamos los siguientes campos
AND
nom_grupo NOT IN ('EMPRESAS RELACIONADAS','TRASPASOS E INTEREMPRESAS')
#AND
#nom_direccion NOT IN ('EXPORTACIÓN ALAMBRES')
#AND
#nom_subdireccion NOT IN ('EXCEDENTES ACEROS','EXCEDENTES ALAMBRES','RESTO ALAMBRES','RESTO INGETEK')
AND
nom_grupo_estadistico1 NOT IN ('CHATARRA','DEACERO POWER','INTERNAS PRODUCCION ','LOGÍSTICA','PALANQUILLA','SEGUNDAS', 'SEGUNDAS')
#AND
#nom_grupo_estadistico2 NOT IN ('SEGUNDAS / EXCEDENTES','CHQ')
#AND
#nom_grupo_estadistico3 NOT IN ('DESPERDICIO','ALAMBRON SEGUNDAS','ALAMBRON EXCEDENTES','ALAMBRON TERCEROS','ALAMBRON OTROS','DERECHO DE VIA','PILOTES','PISO','POLIZAS')
-- Necesitamos que la informacion no sea nula
AND nom_grupo IS NOT NULL
AND nom_direccion IS NOT NULL
AND nom_subdireccion IS NOT NULL
AND nom_grupo_estadistico1 IS NOT NULL
AND nom_grupo_estadistico2 IS NOT NULL
AND nom_grupo_estadistico3 IS NOT NULL
AND  mes_anio IS NOT NULL
GROUP BY 1,2,3,4,5,6,7
ORDER BY 1,2,3,4,5,6,7
"""

client = bigquery.Client(project="demanda-prj-dev")

consulta = query_str

query = client.query(consulta)

pv = query.to_dataframe(create_bqstorage_client=True)

pv.sort_index(inplace=True, ascending=True)

# pv["PVO"] = pv["PVO"].astype(float)

# pv["PV"] = pv["PV"].astype(float)

pv["Toneladas Facturadas"] = pv["Toneladas Facturadas"].astype(float).fillna(np.nan)

pv["toneladas_pvo"] = pv["toneladas_pvo"].astype(float).fillna(np.nan)

pv["toneladas_plan_ventas"] = pv["toneladas_pvo"].astype(float).fillna(np.nan)

# reemplazamos valores muy cercanos a cero por cero
for i in ["Toneladas Facturadas", "toneladas_pvo", "toneladas_plan_ventas"]: 
    pv[i] = np.where(pv[i].abs()<0.00001, 0, pv[i])

pv

c:\Users\adojeda\OneDrive - deacero.com\dev\0_envs\.bigquery\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,MesAnio,NombreGrupo,NombreDireccion,NombreSubdireccion,NombreGrupoEstadistico1,NombreGrupoEstadistico2,NombreGrupoEstadistico3,Toneladas Facturadas,toneladas_pvo,toneladas_plan_ventas
0,2022-01-01,ACEROS,ACEROS MEXICO,EXCEDENTES ACEROS,ALAMBRON,ALAMBRON CONSTRUCCION,ALAMBRON CONSTRUCCION,NaN,0.00,0.00
1,2022-01-01,ACEROS,ACEROS MEXICO,EXCEDENTES ACEROS,MALLAS Y ALAMBRES,CERCOS,CICLONICA GALVANIZADA,NaN,NaN,NaN
2,2022-01-01,ACEROS,ACEROS MEXICO,EXCEDENTES ACEROS,MALLAS Y ALAMBRES,CERCOS ESPECIALIZADOS,MALLA URBANA,NaN,0.04,0.04
3,2022-01-01,ACEROS,ACEROS MEXICO,EXCEDENTES ACEROS,MALLAS Y ALAMBRES,INDUSTRIAL GALV.,A. GALV FINO 18-19,NaN,0.05,0.05
4,2022-01-01,ACEROS,ACEROS MEXICO,EXCEDENTES ACEROS,MALLAS Y ALAMBRES,INDUSTRIAL GALV.,A.GALV. 1/4-14,NaN,0.06,0.06
...,...,...,...,...,...,...,...,...,...,...
71243,2025-12-01,USA,EUA - CANADA,KEY STEEL ACCOUNTS,VARILLA,VARILLA,VARILLA RESTO,NaN,NaN,NaN
71244,2025-12-01,USA,EUA - CANADA,KEY STEEL ACCOUNTS,VARILLA,VARILLA ROLLO,COIL REBAR,NaN,NaN,NaN
71245,2025-12-01,USA,EUA - CANADA,KEY STEEL ACCOUNTS,VARILLA,VARILLA ROLLO,COIL REBAR - CANADA,NaN,NaN,NaN
71246,2025-12-01,USA,EUA - CANADA,KEY STEEL ACCOUNTS,VARILLA,VARILLA ROLLO,SPOOL COIL REBAR,NaN,NaN,NaN


In [9]:
pv["DIR"] = np.where(pv.NombreDireccion.str.contains("EXP") , pv.NombreDireccion, pv.NombreGrupo)

pv["GE12"] = np.where(pv.NombreGrupoEstadistico2.str.contains(" HABI"), pv.NombreGrupoEstadistico2, pv.NombreGrupoEstadistico1)

In [18]:
tab = pv[pv.MesAnio == "2025-08-01"].groupby(['MesAnio', "NombreGrupo","DIR", "GE12"])["toneladas_pvo"].sum().round(0).unstack([-3,-2]).fillna(0)

tab.index = tab.index.droplevel(0)
id_order = ['MALLAS Y ALAMBRES', 'ALAMBRON','VARILLA','PERFILES',  'CABLES', 'INDUSTRIAL FILIALES', 'SOLUCIONES', 'VARILLA HABILITADA', 'PERFILES HABILITADOS']
tab = tab.loc[id_order,:]
tab.loc["total",:] = tab.fillna(0).sum(axis=0)
tab.loc[:,"total"] = tab.fillna(0).sum(axis=1)
tab

NombreGrupo            ACEROS                    ALAMBRES  \
DIR                    ACEROS EXPORTACION ACEROS ALAMBRES   
GE12                                                        
MALLAS Y ALAMBRES      4749.0                0.0  33039.0   
ALAMBRON              29950.0                0.0   2106.0   
VARILLA               20926.0             7500.0  15358.0   
PERFILES              24549.0                0.0    225.0   
CABLES                    0.0                0.0      0.0   
INDUSTRIAL FILIALES       0.0                0.0      0.0   
SOLUCIONES                0.0                0.0      0.0   
VARILLA HABILITADA        0.0                0.0     70.0   
PERFILES HABILITADOS      0.0                0.0      0.0   
total                 80174.0             7500.0  50798.0   

NombreGrupo                               DEACERO SOLUTIONS FILIALES ALAMBRES  \
DIR                  EXPORTACIÓN ALAMBRES DEACERO SOLUTIONS FILIALES ALAMBRES   
GE12                                                                            
MALLAS Y ALAMBRES                  2580.0             778.0            7371.0   
ALAMBRON                              0.0               8.0            6742.0   
VARILLA                             350.0           21737.0            9019.0   
PERFILES                            162.0               0.0               0.0   
CABLES                                0.0               0.0             210.0   
INDUSTRIAL FILIALES                   0.0               0.0            2050.0   
SOLUCIONES                            0.0               0.0               0.0   
VARILLA HABILITADA                    0.0            7353.0               0.0   
PERFILES HABILITADOS                  0.0            4591.0               0.0   
total                              3092.0           34467.0           25392.0   

NombreGrupo               USA     total  
DIR                       USA            
GE12                                     
MALLAS Y ALAMBRES     19684.0   68201.0  
ALAMBRON                300.0   39106.0  
VARILLA                6418.0   81308.0  
PERFILES              15524.0   40460.0  
CABLES                  214.0     424.0  
INDUSTRIAL FILIALES       0.0    2050.0  
SOLUCIONES                0.0       0.0  
VARILLA HABILITADA        0.0    7423.0  
PERFILES HABILITADOS      0.0    4591.0  
total                 42140.0  243563.0

In [11]:
pv

,MesAnio,NombreGrupo,NombreDireccion,NombreSubdireccion,NombreGrupoEstadistico1,NombreGrupoEstadistico2,NombreGrupoEstadistico3,Toneladas Facturadas,toneladas_pvo,toneladas_plan_ventas,DIR,GE12
0,2022-01-01,ACEROS,ACEROS MEXICO,EXCEDENTES ACEROS,ALAMBRON,ALAMBRON CONSTRUCCION,ALAMBRON CONSTRUCCION,NaN,0.00,0.00,ACEROS,ALAMBRON
1,2022-01-01,ACEROS,ACEROS MEXICO,EXCEDENTES ACEROS,MALLAS Y ALAMBRES,CERCOS,CICLONICA GALVANIZADA,NaN,NaN,NaN,ACEROS,MALLAS Y ALAMBRES
2,2022-01-01,ACEROS,ACEROS MEXICO,EXCEDENTES ACEROS,MALLAS Y ALAMBRES,CERCOS ESPECIALIZADOS,MALLA URBANA,NaN,0.04,0.04,ACEROS,MALLAS Y ALAMBRES
3,2022-01-01,ACEROS,ACEROS MEXICO,EXCEDENTES ACEROS,MALLAS Y ALAMBRES,INDUSTRIAL GALV.,A. GALV FINO 18-19,NaN,0.05,0.05,ACEROS,MALLAS Y ALAMBRES
4,2022-01-01,ACEROS,ACEROS MEXICO,EXCEDENTES ACEROS,MALLAS Y ALAMBRES,INDUSTRIAL GALV.,A.GALV. 1/4-14,NaN,0.06,0.06,ACEROS,MALLAS Y ALAMBRES
...,...,...,...,...,...,...,...,...,...,...,...,...
71243,2025-12-01,USA,EUA - CANADA,KEY STEEL ACCOUNTS,VARILLA,VARILLA,VARILLA RESTO,NaN,NaN,NaN,USA,VARILLA
71244,2025-12-01,USA,EUA - CANADA,KEY STEEL ACCOUNTS,VARILLA,VARILLA ROLLO,COIL REBAR,NaN,NaN,NaN,USA,VARILLA
71245,2025-12-01,USA,EUA - CANADA,KEY STEEL ACCOUNTS,VARILLA,VARILLA ROLLO,COIL REBAR - CANADA,NaN,NaN,NaN,USA,VARILLA
71246,2025-12-01,USA,EUA - CANADA,KEY STEEL ACCOUNTS,VARILLA,VARILLA ROLLO,SPOOL COIL REBAR,NaN,NaN,NaN,USA,VARILLA
